# Test Lab for BTE

In [7]:
#!/usr/bin/env python
# coding: utf-8

# Wolf's AlgoHaus Backtester v6.1 - Improved UI
# Compact layout, uniform Helvetica font, adaptive sizing, modern results

import customtkinter as ctk
import tkinter as tk
from tkinter import messagebox, filedialog
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.figure import Figure
import pandas as pd
import numpy as np
from datetime import datetime, timedelta, date
import pathlib
import threading
import queue
import os
import sys
import webbrowser

# ======================================================================
# BACKTESTER UI - IMPROVED COMPACT VERSION
# ======================================================================
class BacktesterUI:
    def __init__(self, master):
        self.master = master
        master.title("⚡ AlgoHaus Backtester v6.1 - Wolf Guzman")

        # ADAPTIVE WINDOW SIZING
        screen_width = master.winfo_screenwidth()
        screen_height = master.winfo_screenheight()
        
        # Compact default size - fits everything efficiently 
        window_width = min(1400, int(screen_width * 0.85))
        window_height = min(900, int(screen_height * 0.85))
        x = (screen_width - window_width) // 2
        y = (screen_height - window_height) // 2
        
        master.geometry(f"{window_width}x{window_height}+{x}+{y}")
        master.minsize(1200, 750)  # Smaller minimum size
        
        # Initialize variables
        default_path = pathlib.Path(r"D:\compressedworld\AlgoHaus\OandaHistoricalData\1MinCharts")
        self.data_folder = default_path if default_path.exists() else pathlib.Path.cwd() / "data"
        self.df = None
        self.current_section = "config"

        self.selected_pair = tk.StringVar(master, value="EUR/USD")
        self.selected_timeframe = tk.StringVar(master, value="1hr")
        self.selected_strategy = tk.StringVar(master, value="vwap_crossover_strategy")
        self.initial_balance = tk.DoubleVar(master, value=10000.0)
        self.leverage = tk.IntVar(master, value=50)
        self.sl_pips = tk.IntVar(master, value=30)
        self.tp_pips = tk.IntVar(master, value=60)
        self.risk_percent = tk.DoubleVar(master, value=1.0)
        self.spread_pips = tk.DoubleVar(master, value=1.5)
        self.slippage_pips = tk.DoubleVar(master, value=0.5)

        today = date.today()
        self.end_date_var = tk.StringVar(master, value=today.strftime("%Y-%m-%d"))
        self.start_date_var = tk.StringVar(master, value=(today - timedelta(days=365)).strftime("%Y-%m-%d"))

        self.status_text = tk.StringVar(master, value="Ready - Wolf Guzman's Trading System v6.1")
        self.metrics_data = {}
        self.trades_df = pd.DataFrame()

        self.setup_ui()
        self.refresh_available_pairs()
        
        # Update pair info after short delay
        self.master.after(500, self.update_pair_info)

    def setup_ui(self):
        # UNIFORM FONT - HELVETICA THROUGHOUT
        self.font_small = ctk.CTkFont(family="Helvetica", size=10)
        self.font_normal = ctk.CTkFont(family="Helvetica", size=11)
        self.font_medium = ctk.CTkFont(family="Helvetica", size=12)
        self.font_large = ctk.CTkFont(family="Helvetica", size=14)
        self.font_header = ctk.CTkFont(family="Helvetica", size=16, weight="bold")

        # Main container
        main_container = ctk.CTkFrame(self.master, corner_radius=0, fg_color="#000000")
        main_container.pack(fill='both', expand=True)

        # COMPACT SIDEBAR - Reduced width
        sidebar = ctk.CTkFrame(main_container, corner_radius=0, fg_color="#000000", width=220)  # Reduced from 260
        sidebar.pack(side='left', fill='y')
        sidebar.pack_propagate(False)

        # COMPACT LOGO
        logo_frame = ctk.CTkFrame(sidebar, corner_radius=0, fg_color="transparent")
        logo_frame.pack(fill='x', padx=15, pady=(15, 25))  # Reduced padding

        ctk.CTkLabel(
            logo_frame,
            text="⚡ algoHaus",
            font=ctk.CTkFont(family="Helvetica", size=18, weight="bold"),  # Smaller
            text_color="#029cff",
            anchor="w"
        ).pack(anchor='w')

        ctk.CTkLabel(
            logo_frame,
            text="Compact Trading Engine",
            font=self.font_small,
            text_color="#6e7681",
            anchor="w"
        ).pack(anchor='w', pady=(2, 0))

        # COMPACT NAVIGATION
        nav_frame = ctk.CTkFrame(sidebar, corner_radius=0, fg_color="transparent")
        nav_frame.pack(fill='x', padx=10, pady=(0, 20))  # Reduced padding

        self.nav_buttons = {}

        self.nav_buttons['config'] = self.create_nav_button(
            nav_frame, "⚙ Config", "config", selected=True
        )

        self.nav_buttons['strategy'] = self.create_nav_button(
            nav_frame, "📊 Strategy", "strategy"
        )

        self.nav_buttons['account'] = self.create_nav_button(
            nav_frame, "💰 Account", "account"
        )

        self.nav_buttons['results'] = self.create_nav_button(
            nav_frame, "📈 Results", "results"
        )

        # GRAY STYLED BUTTONS (like active tabs/results)
        self.run_btn = ctk.CTkButton(
            sidebar,
            text="▶ RUN BACKTEST",
            font=self.font_medium,
            fg_color="#21262d",           # Gray like active tabs
            hover_color="#30363d",        # Darker gray on hover
            text_color="#e6edf3",         # Light text
            height=40,                    # Reduced height
            corner_radius=6,
            command=self.start_backtest_thread
        )
        self.run_btn.pack(fill='x', padx=10, pady=(15, 8))

        # GRAY REPORT BUTTON
        self.report_button = ctk.CTkButton(
            sidebar,
            text="📄 Generate Report",
            font=self.font_normal,
            fg_color="#21262d",           # Gray like active tabs
            hover_color="#30363d",
            text_color="#8b949e",         # Muted text when disabled
            height=35,                    # Reduced height
            corner_radius=6,
            command=self.generate_report,
            state="disabled"
        )
        self.report_button.pack(fill='x', padx=10, pady=(0, 8))

        # COMPACT UTILITY BUTTONS
        utility_container = ctk.CTkFrame(sidebar, fg_color="transparent")
        utility_container.pack(fill='x', padx=10, pady=(5, 15))

        ctk.CTkButton(
            utility_container,
            text="🗑️ Cache",
            font=self.font_small,
            fg_color="#21262d",
            hover_color="#30363d",
            text_color="#8b949e",
            height=28,                    # Compact
            corner_radius=6,
            command=self.clear_cache
        ).pack(side='left', expand=True, fill='x', padx=(0, 3))

        ctk.CTkButton(
            utility_container,
            text="📊 Export",
            font=self.font_small,
            fg_color="#21262d",
            hover_color="#30363d",
            text_color="#8b949e",
            height=28,                    # Compact
            corner_radius=6,
            command=self.export_to_csv
        ).pack(side='left', expand=True, fill='x', padx=(3, 0))

        # ADAPTIVE CONTENT AREA
        content_area = ctk.CTkFrame(main_container, corner_radius=0, fg_color="#000000")
        content_area.pack(side='right', fill='both', expand=True)

        self.content_frame = ctk.CTkFrame(content_area, fg_color="transparent")
        self.content_frame.pack(fill='both', expand=True, padx=25, pady=20)  # Reduced padding

        self.sections = {}
        self.create_config_section()
        self.create_strategy_section()
        self.create_account_section()
        self.create_results_section()

        self.show_section('config')

        # COMPACT PROGRESS BAR
        self.progress_container = ctk.CTkFrame(content_area, fg_color="#000000", corner_radius=8)

        self.progress_label = ctk.CTkLabel(
            self.progress_container,
            text="",
            font=self.font_small,
            text_color="#8b949e",
            anchor="w"
        )
        self.progress_label.pack(fill='x', padx=12, pady=(8, 4))

        self.progress_bar = ctk.CTkProgressBar(
            self.progress_container,
            mode="determinate",
            height=4,                     # Thinner progress bar
            corner_radius=2,
            fg_color="#21262d",
            progress_color="#238636"
        )
        self.progress_bar.pack(fill='x', padx=12, pady=(0, 8))
        self.progress_bar.set(0)

        # COMPACT STATUS BAR
        status_bar = ctk.CTkFrame(self.master, corner_radius=0, height=30, fg_color="#000000")
        status_bar.pack(side='bottom', fill='x')

        self.status_label = ctk.CTkLabel(
            status_bar,
            textvariable=self.status_text,
            font=self.font_small,
            text_color="#8b949e",
            anchor="w"
        )
        self.status_label.pack(side='left', padx=20, pady=8)

    def create_nav_button(self, parent, text, section_id, selected=False):
        btn = ctk.CTkButton(
            parent,
            text=text,
            font=self.font_normal,
            fg_color="#21262d" if selected else "transparent",
            hover_color="#30363d" if selected else "#21262d",
            text_color="#e6edf3" if selected else "#8b949e",
            anchor="w",
            height=32,                    # Compact height
            corner_radius=6,
            command=lambda: self.show_section(section_id)
        )
        btn.pack(fill='x', pady=1)        # Reduced spacing
        return btn

    def update_nav_selection(self, selected_section):
        for section_id, btn in self.nav_buttons.items():
            if section_id == selected_section:
                btn.configure(fg_color="#21262d", text_color="#e6edf3")
            else:
                btn.configure(fg_color="transparent", text_color="#8b949e")

    def show_section(self, section_id):
        self.current_section = section_id
        self.update_nav_selection(section_id)

        for sec_id, sec_frame in self.sections.items():
            if sec_id == section_id:
                sec_frame.pack(fill='both', expand=True)
            else:
                sec_frame.pack_forget()

    def create_config_section(self):
        section = ctk.CTkFrame(self.content_frame, fg_color="transparent")
        self.sections['config'] = section

        ctk.CTkLabel(
            section,
            text="CONFIGURATION",
            font=self.font_header,
            text_color="#e6edf3",
            anchor="w"
        ).pack(anchor='w', pady=(0, 15))

        content = ctk.CTkFrame(section, fg_color="#000000", corner_radius=8)
        content.pack(fill='both', expand=True)

        inner = ctk.CTkFrame(content, fg_color="transparent")
        inner.pack(fill='both', expand=True, padx=20, pady=20)

        # Data Folder
        self.create_section_header(inner, "Data Source")
        folder_frame = ctk.CTkFrame(inner, fg_color="#21262d", corner_radius=6)
        folder_frame.pack(fill='x', pady=(0, 15))

        folder_inner = ctk.CTkFrame(folder_frame, fg_color="transparent")
        folder_inner.pack(fill='x', padx=12, pady=10)

        ctk.CTkLabel(
            folder_inner,
            text="Data Folder",
            font=self.font_normal,
            text_color="#8b949e",
            width=80,
            anchor="w"
        ).pack(side='left')

        folder_display = str(self.data_folder)
        if len(folder_display) > 45:
            folder_display = "..." + folder_display[-42:]

        self.folder_label = ctk.CTkLabel(
            folder_inner,
            text=folder_display,
            font=self.font_small,
            text_color="#e6edf3",
            anchor="w"
        )
        self.folder_label.pack(side='left', fill='x', expand=True, padx=10)

        ctk.CTkButton(
            folder_inner,
            text="Browse",
            command=self.select_data_folder,
            width=60,
            height=26,
            font=self.font_small,
            fg_color="#30363d",
            hover_color="#484f58",
            text_color="#e6edf3",
            corner_radius=4
        ).pack(side='right')

        # Trading Pair
        self.create_section_header(inner, "Trading Pair")
        self.pair_combo = self.create_compact_input(inner, "Pair", self.selected_pair, is_combobox=True, values=["EUR/USD"])

        # COMPACT Pair Info
        self.pair_info_frame = ctk.CTkFrame(inner, fg_color="#21262d", corner_radius=6)
        self.pair_info_frame.pack(fill='x', pady=(8, 15))

        self.pair_info_label = ctk.CTkLabel(
            self.pair_info_frame,
            text='Select a pair to view details...',
            font=self.font_small,
            text_color="#8b949e",
            anchor="nw",
            justify="left"
        )
        self.pair_info_label.pack(fill='both', padx=12, pady=10)

        self.selected_pair.trace('w', self.update_pair_info)

        # Timeframe & Dates
        self.create_section_header(inner, "Time Period")
        self.create_compact_input(inner, "Timeframe", self.selected_timeframe, is_combobox=True, 
                               values=["1min", "5min", "15min", "1hr", "1Day"])
        self.create_compact_input(inner, "Start Date", self.start_date_var)
        self.create_compact_input(inner, "End Date", self.end_date_var)

    def create_strategy_section(self):
        section = ctk.CTkFrame(self.content_frame, fg_color="transparent")
        self.sections['strategy'] = section

        ctk.CTkLabel(
            section,
            text="STRATEGY & RISK",
            font=self.font_header,
            text_color="#e6edf3",
            anchor="w"
        ).pack(anchor='w', pady=(0, 15))

        content = ctk.CTkFrame(section, fg_color="#000000", corner_radius=8)
        content.pack(fill='both', expand=True)

        inner = ctk.CTkFrame(content, fg_color="transparent")
        inner.pack(fill='both', expand=True, padx=20, pady=20)

        self.create_section_header(inner, "Trading Strategy")
        strategies = ["vwap_crossover_strategy", "opening_range_strategy", "bollinger_band_reversion_strategy"]

        self.create_compact_input(inner, "Strategy", self.selected_strategy, is_combobox=True, values=strategies)

        self.create_section_header(inner, "Risk Management")
        self.create_compact_input(inner, "Stop Loss (pips)", self.sl_pips)
        self.create_compact_input(inner, "Take Profit (pips)", self.tp_pips)

        self.create_section_header(inner, "Execution Costs")
        self.create_compact_input(inner, "Spread (pips)", self.spread_pips)
        self.create_compact_input(inner, "Slippage (pips)", self.slippage_pips)

    def create_account_section(self):
        section = ctk.CTkFrame(self.content_frame, fg_color="transparent")
        self.sections['account'] = section

        ctk.CTkLabel(
            section,
            text="ACCOUNT SETTINGS",
            font=self.font_header,
            text_color="#e6edf3",
            anchor="w"
        ).pack(anchor='w', pady=(0, 15))

        content = ctk.CTkFrame(section, fg_color="#000000", corner_radius=8)
        content.pack(fill='both', expand=True)

        inner = ctk.CTkFrame(content, fg_color="transparent")
        inner.pack(fill='both', expand=True, padx=20, pady=20)

        self.create_section_header(inner, "Capital & Leverage")
        self.create_compact_input(inner, "Initial Balance ($)", self.initial_balance)
        self.create_compact_input(inner, "Leverage", self.leverage, is_combobox=True, 
                               values=[str(x) for x in [1, 10, 20, 30, 50, 100, 200, 500]])

        self.create_section_header(inner, "Position Sizing")
        self.create_compact_input(inner, "Risk % per Trade", self.risk_percent)

    def create_results_section(self):
        section = ctk.CTkFrame(self.content_frame, fg_color="transparent")
        self.sections['results'] = section

        # MAIN RESULTS CONTAINER - Professional Layout
        main_results_container = ctk.CTkFrame(section, fg_color="transparent")
        main_results_container.pack(fill='both', expand=True)

        # TOP SECTION - Summary + Metrics Table (Like Reference Image)
        top_section = ctk.CTkFrame(main_results_container, fg_color="#000000", corner_radius=8)
        top_section.pack(fill='x', pady=(0, 15))

        # Create two-column layout: Summary Left, Metrics Table Right
        content_row = ctk.CTkFrame(top_section, fg_color="transparent")
        content_row.pack(fill='both', expand=True, padx=15, pady=15)

        # LEFT: SUMMARY SECTION (like reference image)
        summary_container = ctk.CTkFrame(content_row, fg_color="#21262d", corner_radius=8, width=300)
        summary_container.pack(side='left', fill='y', padx=(0, 15))
        summary_container.pack_propagate(False)

        # Summary header
        summary_header = ctk.CTkLabel(
            summary_container,
            text="SUMMARY",
            font=ctk.CTkFont(family="Helvetica", size=13, weight="bold"),
            text_color="#8b949e",
            anchor="w"
        )
        summary_header.pack(anchor='w', padx=15, pady=(15, 8))

        # Summary content
        self.summary_textbox = ctk.CTkTextbox(
            summary_container,
            height=180,
            corner_radius=6,
            font=ctk.CTkFont(family="Helvetica", size=13, weight="bold"),
            fg_color="#21262d",
            text_color="#e6edf3",
            border_width=0
        )
        self.summary_textbox.pack(fill='both', expand=True, padx=15, pady=(0, 15))

        # RIGHT: METRICS TABLE (like reference image)
        metrics_container = ctk.CTkFrame(content_row, fg_color="#21262d", corner_radius=8)
        metrics_container.pack(side='right', fill='both', expand=True)

        # Create scrollable frame for metrics table
        self.metrics_table_frame = ctk.CTkScrollableFrame(
            metrics_container,
            height=200,
            corner_radius=6,
            fg_color="#21262d",
            scrollbar_button_color="#30363d",
            scrollbar_button_hover_color="#484f58"
        )
        self.metrics_table_frame.pack(fill='both', expand=True, padx=15, pady=15)

        # CHARTS SECTION (Bottom - Already Perfect)
        charts_container = ctk.CTkFrame(main_results_container, fg_color="transparent")
        charts_container.pack(fill='both', expand=True)

        self.charts_display = ctk.CTkFrame(charts_container, fg_color="#000000", corner_radius=8)
        self.charts_display.pack(fill='both', expand=True)

    def update_results_ui(self, summary, metrics, trades_df):
        """Updated results UI with professional dashboard layout matching reference"""
        self.trades_df = trades_df
        self.metrics_data = metrics

        # Update SUMMARY section
        self.summary_textbox.delete("0.0", "end")
        lines = summary.split('\n')
        for i, line in enumerate(lines):
            if i > 0:
                self.summary_textbox.insert("end", "\n")

            # Color-code the returns line
            if line.startswith("RETURNS:"):
                try:
                    returns_value = float(line.split(':')[1].strip().replace('%', '').replace('+', ''))
                    color = "#00ff88" if returns_value >= 0 else "#ff3366"

                    start_pos = self.summary_textbox.index("end-1c")
                    self.summary_textbox.insert("end", line)
                    end_pos = self.summary_textbox.index("end-1c")
                    self.summary_textbox.tag_add("returns_color", start_pos, end_pos)
                    self.summary_textbox.tag_config("returns_color", foreground=color)
                except:
                    self.summary_textbox.insert("end", line)
            else:
                self.summary_textbox.insert("end", line)

        # Clear and update METRICS TABLE
        for widget in self.metrics_table_frame.winfo_children():
            widget.destroy()

        if metrics:
            self.create_professional_metrics_table(metrics)

        # Clear and update CHARTS
        for widget in self.charts_display.winfo_children():
            widget.destroy()

        if not trades_df.empty:
            self.create_modern_charts(trades_df)

        # Enable report button
        self.report_button.configure(
            state="normal",
            text_color="#e6edf3",
            fg_color="#21262d",
            hover_color="#30363d"
        )

        # Auto-switch to results
        self.show_section('results')

    def create_professional_metrics_table(self, metrics):
        """Create professional metrics table matching the reference image layout"""
        
        # TABLE HEADERS (matching reference image)
        header_frame = ctk.CTkFrame(self.metrics_table_frame, fg_color="#30363d", corner_radius=6, height=35)
        header_frame.pack(fill='x', pady=(0, 5))
        header_frame.pack_propagate(False)

        # Header columns (5 columns like reference)
        headers = ["METRIC", "VALUE", "METRIC", "VALUE", "STATUS"]
        header_widths = [140, 80, 140, 80, 80]

        for i, (header, width) in enumerate(zip(headers, header_widths)):
            ctk.CTkLabel(
                header_frame,
                text=header,
                font=ctk.CTkFont(family="Helvetica", size=10, weight="bold"),
                text_color="#8b949e",
                width=width,
                anchor="w" if i == 0 or i == 2 else "center"
            ).pack(side='left', padx=8, pady=8)

        # METRICS DATA (organized in pairs like reference image)
        metrics_pairs = [
            # Row 1
            ("Total Trades", metrics.get('Total Trades', 0), 
             "Win Rate", f"{metrics.get('Win Rate (%)', 0):.1f}%",
             "✓" if metrics.get('Win Rate (%)', 0) > 50 else "⚠"),
            
            # Row 2  
            ("Profit Factor", f"{metrics.get('Profit Factor', 0):.2f}",
             "Max Drawdown", f"{metrics.get('Max Drawdown (%)', 0):.1f}%",
             "✓" if metrics.get('Profit Factor', 0) > 1.5 else "⚠"),
            
            # Row 3
            ("Sharpe Ratio", f"{metrics.get('Sharpe Ratio', 0):.2f}",
             "Sortino Ratio", f"{metrics.get('Sortino Ratio', 0):.2f}",
             "✓" if metrics.get('Sharpe Ratio', 0) > 1 else "⚠"),
            
            # Row 4
            ("Avg Win", f"${metrics.get('Avg Win ($)', 0):,.0f}",
             "Avg Loss", f"${metrics.get('Avg Loss ($)', 0):,.0f}",
             "✓" if abs(metrics.get('Avg Win ($)', 0)) > abs(metrics.get('Avg Loss ($)', 0)) else "⚠"),
            
            # Row 5
            ("Largest Win", f"${metrics.get('Largest Win ($)', 0):,.0f}",
             "Largest Loss", f"${metrics.get('Largest Loss ($)', 0):,.0f}",
             "✓"),
            
            # Row 6
            ("Max Win Streak", f"{metrics.get('Max Consecutive Wins', 0)}",
             "Max Loss Streak", f"{metrics.get('Max Consecutive Losses', 0)}",
             "✓" if metrics.get('Max Consecutive Wins', 0) > metrics.get('Max Consecutive Losses', 0) else "⚠"),
        ]

        # Create table rows
        for i, (metric1, value1, metric2, value2, status) in enumerate(metrics_pairs):
            # Alternate row colors
            row_color = "#21262d" if i % 2 == 0 else "#1a1a1a"
            row_frame = ctk.CTkFrame(self.metrics_table_frame, fg_color=row_color, corner_radius=4, height=32)
            row_frame.pack(fill='x', pady=1)
            row_frame.pack_propagate(False)

            # Column 1: First Metric Name
            ctk.CTkLabel(
                row_frame,
                text=metric1,
                font=self.font_normal,
                text_color="#e6edf3",
                width=140,
                anchor="w"
            ).pack(side='left', padx=8, pady=6)

            # Column 2: First Value
            value1_color = "#00ff88" if any(x in metric1.lower() for x in ['win', 'profit', 'sharpe', 'sortino']) and ('$' in value1 or float(value1.replace('$', '').replace(',', '').replace('%', '')) > 0) else "#e6edf3"
            if 'loss' in metric1.lower() or 'drawdown' in metric1.lower():
                value1_color = "#ff3366"
            
            ctk.CTkLabel(
                row_frame,
                text=value1,
                font=ctk.CTkFont(family="Helvetica", size=11, weight="bold"),
                text_color=value1_color,
                width=80,
                anchor="center"
            ).pack(side='left', padx=8, pady=6)

            # Column 3: Second Metric Name
            ctk.CTkLabel(
                row_frame,
                text=metric2,
                font=self.font_normal,
                text_color="#e6edf3",
                width=140,
                anchor="w"
            ).pack(side='left', padx=8, pady=6)

            # Column 4: Second Value
            value2_color = "#00ff88" if any(x in metric2.lower() for x in ['win', 'profit', 'sharpe', 'sortino']) and ('$' in value2 or '%' in value2) else "#e6edf3"
            if 'loss' in metric2.lower() or 'drawdown' in metric2.lower():
                value2_color = "#ff3366"
            
            ctk.CTkLabel(
                row_frame,
                text=value2,
                font=ctk.CTkFont(family="Helvetica", size=11, weight="bold"),
                text_color=value2_color,
                width=80,
                anchor="center"
            ).pack(side='left', padx=8, pady=6)

            # Column 5: Status
            status_color = "#00ff88" if status == "✓" else "#ffcc00"
            ctk.CTkLabel(
                row_frame,
                text=status,
                font=ctk.CTkFont(family="Helvetica", size=14, weight="bold"),
                text_color=status_color,
                width=80,
                anchor="center"
            ).pack(side='left', padx=8, pady=6)

    def create_section_header(self, parent, text):
        ctk.CTkLabel(
            parent,
            text=text,
            font=self.font_medium,
            text_color="#8b949e",
            anchor="w"
        ).pack(anchor='w', pady=(10, 6))

    def create_compact_input(self, parent, label_text, variable, is_combobox=False, values=None):
        """COMPACT input with smaller widths and uniform font - BLACK backgrounds"""
        frame = ctk.CTkFrame(parent, fg_color="transparent")
        frame.pack(fill='x', pady=4)

        ctk.CTkLabel(
            frame,
            text=label_text,
            font=self.font_normal,
            text_color="#8b949e",
            width=100,                    # Reduced label width
            anchor="w"
        ).pack(side='left')

        if is_combobox:
            widget = ctk.CTkComboBox(
                frame,
                variable=variable,
                values=values or [],
                font=self.font_normal,
                fg_color="#000000",           # Pure black background
                border_color="#21262d",       # Dark gray border
                button_color="#21262d",       # Dark gray button
                button_hover_color="#30363d", # Slightly lighter on hover
                text_color="#e6edf3",         # Light text
                dropdown_fg_color="#000000",  # Pure black dropdown menu
                dropdown_hover_color="#21262d", # Dark gray hover in dropdown
                dropdown_text_color="#e6edf3", # Light text in dropdown
                border_width=1,               # Thin border
                corner_radius=4,
                height=28,                    # Compact height
                width=200                     # Reduced width
            )
        else:
            widget = ctk.CTkEntry(
                frame,
                textvariable=variable,
                font=self.font_normal,
                fg_color="#000000",           # Pure black background  
                border_color="#21262d",       # Dark gray border
                text_color="#e6edf3",         # Light text
                border_width=1,               # Thin border
                corner_radius=4,
                height=28,                    # Compact height
                width=200                     # Reduced width
            )

        widget.pack(side='left', padx=(10, 0))
        return widget

    def update_results_ui(self, summary, metrics, trades_df):
        """Updated results UI with professional dashboard layout matching reference"""
        self.trades_df = trades_df
        self.metrics_data = metrics

        # Update SUMMARY section
        self.summary_textbox.delete("0.0", "end")
        lines = summary.split('\n')
        for i, line in enumerate(lines):
            if i > 0:
                self.summary_textbox.insert("end", "\n")

            # Color-code the returns line
            if line.startswith("RETURNS:"):
                try:
                    returns_value = float(line.split(':')[1].strip().replace('%', '').replace('+', ''))
                    color = "#00ff88" if returns_value >= 0 else "#ff3366"

                    start_pos = self.summary_textbox.index("end-1c")
                    self.summary_textbox.insert("end", line)
                    end_pos = self.summary_textbox.index("end-1c")
                    self.summary_textbox.tag_add("returns_color", start_pos, end_pos)
                    self.summary_textbox.tag_config("returns_color", foreground=color)
                except:
                    self.summary_textbox.insert("end", line)
            else:
                self.summary_textbox.insert("end", line)

        # Clear and update METRICS TABLE
        for widget in self.metrics_table_frame.winfo_children():
            widget.destroy()

        if metrics:
            self.create_professional_metrics_table(metrics)

        # Clear and update CHARTS
        for widget in self.charts_display.winfo_children():
            widget.destroy()

        if not trades_df.empty:
            self.create_modern_charts(trades_df)

        # Enable report button
        self.report_button.configure(
            state="normal",
            text_color="#e6edf3",
            fg_color="#21262d",
            hover_color="#30363d"
        )

        # Auto-switch to results
        self.show_section('results')

    def create_metrics_cards(self, metrics):
        """Create horizontal metrics cards like reference image"""
        # Key metrics to display as cards
        key_metrics = [
            ('Total Return', f"{metrics.get('Total Return (%)', 0):.1f}%", 
             '#3fb950' if metrics.get('Total Return (%)', 0) > 0 else '#f85149'),
            ('Win Rate', f"{metrics.get('Win Rate (%)', 0):.1f}%", 
             '#3fb950' if metrics.get('Win Rate (%)', 0) > 50 else '#f85149'),
            ('Profit Factor', f"{metrics.get('Profit Factor', 0):.2f}", 
             '#3fb950' if metrics.get('Profit Factor', 0) > 1 else '#f85149'),
            ('Max Drawdown', f"{metrics.get('Max Drawdown (%)', 0):.1f}%", '#f85149')
        ]

        cards_frame = ctk.CTkFrame(self.metrics_cards_container, fg_color="transparent")
        cards_frame.pack(fill='x')

        for i, (label, value, color) in enumerate(key_metrics):
            card = ctk.CTkFrame(cards_frame, fg_color="#21262d", corner_radius=6, width=120, height=70)
            card.pack(side='left', padx=5, pady=5)
            card.pack_propagate(False)

            # Value
            ctk.CTkLabel(
                card,
                text=value,
                font=ctk.CTkFont(family="Helvetica", size=16, weight="bold"),
                text_color=color,
                anchor="center"
            ).pack(expand=True, pady=(8, 0))

            # Label
            ctk.CTkLabel(
                card,
                text=label,
                font=self.font_small,
                text_color="#8b949e",
                anchor="center"
            ).pack(pady=(0, 8))

    def create_modern_charts(self, trades_df):
        """Create modern dashboard with proper black backgrounds and no white borders"""
        # Set matplotlib to use complete dark theme with no white borders
        plt.style.use('dark_background')
        plt.rcParams['figure.facecolor'] = '#000000'
        plt.rcParams['axes.facecolor'] = '#000000'
        plt.rcParams['axes.edgecolor'] = '#000000'
        plt.rcParams['figure.edgecolor'] = '#000000'
        plt.rcParams['savefig.facecolor'] = '#000000'
        plt.rcParams['savefig.edgecolor'] = '#000000'
        
        # Create figure with black background and NO white borders
        fig = Figure(figsize=(12, 7), facecolor='#000000', edgecolor='#000000')
        fig.patch.set_facecolor('#000000')
        fig.subplots_adjust(left=0.08, right=0.96, top=0.92, bottom=0.12, hspace=0.3, wspace=0.25)

        # Professional colors matching dashboard
        BG_COLOR = '#000000'
        GRID_COLOR = '#21262d'
        TEXT_COLOR = '#8b949e'
        ACCENT_GREEN = '#00ff88'
        ACCENT_RED = '#ff3366'
        ACCENT_BLUE = '#00d4ff'

        # Chart 1: EQUITY CURVE (Top Left) 
        ax1 = fig.add_subplot(2, 2, 1, facecolor=BG_COLOR)
        ax1.set_facecolor(BG_COLOR)
        
        if not trades_df.empty:
            dates = trades_df['exit_time']
            equity = trades_df['balance']

            # Main equity line
            ax1.plot(dates, equity, color=ACCENT_GREEN, linewidth=2.5, alpha=0.9)
            ax1.fill_between(dates, equity, self.initial_balance.get(), 
                           alpha=0.2, color=ACCENT_GREEN, interpolate=True)
            
            # Initial balance reference line
            ax1.axhline(y=self.initial_balance.get(), color=TEXT_COLOR, 
                       linestyle='--', alpha=0.6, linewidth=1)
            
            ax1.set_title('Equity Curve', fontsize=12, color='#ffffff', 
                         fontfamily='Helvetica', fontweight='bold', pad=15)
            ax1.tick_params(colors=TEXT_COLOR, labelsize=9)
            ax1.grid(True, alpha=0.2, color=GRID_COLOR, linewidth=0.8)
            
            # Remove all spines (borders)
            for spine in ax1.spines.values():
                spine.set_color('#000000')
                spine.set_linewidth(0)

        # Chart 2: DRAWDOWN (Top Right)
        ax2 = fig.add_subplot(2, 2, 2, facecolor=BG_COLOR)
        ax2.set_facecolor(BG_COLOR)
        
        if not trades_df.empty:
            cumulative_pnl = trades_df['monetary_pnl'].cumsum()
            equity_curve = self.initial_balance.get() + cumulative_pnl
            running_max = equity_curve.expanding().max()
            drawdown = ((equity_curve - running_max) / running_max) * 100

            ax2.fill_between(dates, drawdown, 0, color=ACCENT_RED, alpha=0.4, interpolate=True)
            ax2.plot(dates, drawdown, color=ACCENT_RED, linewidth=2.5, alpha=0.9)
            
            ax2.set_title('Drawdown', fontsize=12, color='#ffffff', 
                         fontfamily='Helvetica', fontweight='bold', pad=15)
            ax2.tick_params(colors=TEXT_COLOR, labelsize=9)
            ax2.grid(True, alpha=0.2, color=GRID_COLOR, linewidth=0.8)
            ax2.set_ylabel('Drawdown (%)', color=TEXT_COLOR, fontsize=10, fontfamily='Helvetica')
            
            # Remove all spines
            for spine in ax2.spines.values():
                spine.set_color('#000000')
                spine.set_linewidth(0)

        # Chart 3: P&L HISTOGRAM (Bottom Left)
        ax3 = fig.add_subplot(2, 2, 3, facecolor=BG_COLOR)
        ax3.set_facecolor(BG_COLOR)
        
        if not trades_df.empty:
            returns = trades_df['monetary_pnl']
            
            # Create histogram with no white edges
            n, bins, patches = ax3.hist(returns, bins=25, alpha=0.8, 
                                      edgecolor='#000000', linewidth=0)

            # Color bars based on positive/negative
            for i, patch in enumerate(patches):
                bin_center = (bins[i] + bins[i+1]) / 2
                if bin_center < 0:
                    patch.set_facecolor(ACCENT_RED)
                    patch.set_alpha(0.7)
                else:
                    patch.set_facecolor(ACCENT_GREEN)
                    patch.set_alpha(0.7)
                patch.set_edgecolor('#000000')

            # Mean line
            mean_return = returns.mean()
            ax3.axvline(x=mean_return, color=ACCENT_BLUE, linestyle='--', 
                       linewidth=2, alpha=0.8, label=f'Mean: ${mean_return:.0f}')
            ax3.axvline(x=0, color='#ffffff', linestyle='-', 
                       linewidth=1, alpha=0.5)

            ax3.set_title('P&L Distribution (Histogram)', fontsize=12, color='#ffffff', 
                         fontfamily='Helvetica', fontweight='bold', pad=15)
            ax3.tick_params(colors=TEXT_COLOR, labelsize=9)
            ax3.grid(True, alpha=0.2, color=GRID_COLOR, linewidth=0.8)
            ax3.set_xlabel('P&L ($)', color=TEXT_COLOR, fontsize=10, fontfamily='Helvetica')
            ax3.set_ylabel('Frequency', color=TEXT_COLOR, fontsize=10, fontfamily='Helvetica')
            ax3.legend(loc='upper right', fontsize=9, facecolor='#21262d', 
                      edgecolor='#000000', labelcolor='#ffffff')
            
            # Remove all spines
            for spine in ax3.spines.values():
                spine.set_color('#000000')
                spine.set_linewidth(0)

        # Chart 4: Monthly Returns Bar Chart (Bottom Right)
        ax4 = fig.add_subplot(2, 2, 4, facecolor=BG_COLOR)
        ax4.set_facecolor(BG_COLOR)
        
        if not trades_df.empty:
            # Group by month for monthly returns
            trades_df_copy = trades_df.copy()
            trades_df_copy['month'] = trades_df_copy['exit_time'].dt.to_period('M')
            monthly_pnl = trades_df_copy.groupby('month')['monetary_pnl'].sum()
            
            if len(monthly_pnl) > 0:
                months = [str(m) for m in monthly_pnl.index]
                values = monthly_pnl.values
                
                # Color bars based on positive/negative
                colors = [ACCENT_GREEN if v >= 0 else ACCENT_RED for v in values]
                
                bars = ax4.bar(range(len(months)), values, color=colors, 
                             alpha=0.8, edgecolor='#000000', linewidth=0)
                
                # Zero line
                ax4.axhline(y=0, color='#ffffff', linestyle='-', linewidth=1, alpha=0.5)
                
                ax4.set_title('Monthly Returns', fontsize=12, color='#ffffff', 
                             fontfamily='Helvetica', fontweight='bold', pad=15)
                ax4.tick_params(colors=TEXT_COLOR, labelsize=8)
                ax4.grid(True, alpha=0.2, color=GRID_COLOR, linewidth=0.8, axis='y')
                ax4.set_ylabel('Monthly P&L ($)', color=TEXT_COLOR, fontsize=10, fontfamily='Helvetica')
                
                # Rotate x-axis labels if too many months
                if len(months) > 6:
                    ax4.set_xticks(range(0, len(months), max(1, len(months)//6)))
                    ax4.set_xticklabels([months[i] for i in range(0, len(months), max(1, len(months)//6))], 
                                       rotation=45, ha='right')
                else:
                    ax4.set_xticks(range(len(months)))
                    ax4.set_xticklabels(months, rotation=45, ha='right')
            
            # Remove all spines
            for spine in ax4.spines.values():
                spine.set_color('#000000')
                spine.set_linewidth(0)

        # Create canvas with black background
        canvas = FigureCanvasTkAgg(fig, master=self.charts_display)
        canvas.draw()
        widget = canvas.get_tk_widget()
        widget.configure(bg='#000000', highlightthickness=0, bd=0)
        widget.pack(fill='both', expand=True, padx=10, pady=10)

    def create_advanced_stats_table(self, metrics):
        """Create advanced statistics table with high-end trading metrics"""
        # Clear previous content
        for widget in self.advanced_stats_frame.winfo_children():
            widget.destroy()

        if not metrics:
            return

        # Advanced metrics for professional traders
        advanced_metrics = [
            ('Sharpe Ratio', metrics.get('Sharpe Ratio', 0)),
            ('Sortino Ratio', metrics.get('Sortino Ratio', 0)), 
            ('Max Consecutive Wins', metrics.get('Max Consecutive Wins', 0)),
            ('Max Consecutive Losses', metrics.get('Max Consecutive Losses', 0)),
            ('Largest Win', f"${metrics.get('Largest Win ($)', 0):,.2f}"),
            ('Largest Loss', f"${metrics.get('Largest Loss ($)', 0):,.2f}"),
            ('Average Win', f"${metrics.get('Avg Win ($)', 0):,.2f}"),
            ('Average Loss', f"${metrics.get('Avg Loss ($)', 0):,.2f}"),
            ('Risk-Reward Ratio', f"{abs(metrics.get('Avg Win ($)', 0) / metrics.get('Avg Loss ($)', 1)):.2f}:1" if metrics.get('Avg Loss ($)', 0) != 0 else "N/A"),
            ('Expectancy per Trade', f"${(metrics.get('Win Rate (%)', 0)/100 * metrics.get('Avg Win ($)', 0)) + ((100-metrics.get('Win Rate (%)', 0))/100 * -metrics.get('Avg Loss ($)', 0)):.2f}")
        ]

        # Create table header
        header_frame = ctk.CTkFrame(self.advanced_stats_frame, fg_color="#30363d", corner_radius=4)
        header_frame.pack(fill='x', pady=(0, 5))

        ctk.CTkLabel(header_frame, text="Metric", font=ctk.CTkFont(family="Helvetica", size=11, weight="bold"), 
                    text_color="#e6edf3", anchor="w", width=200).pack(side='left', padx=10, pady=8)
        ctk.CTkLabel(header_frame, text="Value", font=ctk.CTkFont(family="Helvetica", size=11, weight="bold"), 
                    text_color="#e6edf3", anchor="w", width=150).pack(side='left', padx=10, pady=8)

        # Create rows
        for i, (metric_name, value) in enumerate(advanced_metrics):
            row_color = "#21262d" if i % 2 == 0 else "#1a1a1a"
            row_frame = ctk.CTkFrame(self.advanced_stats_frame, fg_color=row_color, corner_radius=4)
            row_frame.pack(fill='x', pady=1)

            ctk.CTkLabel(row_frame, text=metric_name, font=self.font_normal, 
                        text_color="#e6edf3", anchor="w", width=200).pack(side='left', padx=10, pady=6)
            
            # Color-code certain values
            if isinstance(value, (int, float)) and not isinstance(value, str):
                value_text = f"{value:.2f}"
                if 'Ratio' in metric_name and value > 1:
                    text_color = "#3fb950"
                elif 'Loss' in metric_name:
                    text_color = "#f85149"
                elif value > 0:
                    text_color = "#3fb950"
                else:
                    text_color = "#f85149"
            else:
                value_text = str(value)
                text_color = "#e6edf3"

            ctk.CTkLabel(row_frame, text=value_text, font=self.font_normal, 
                        text_color=text_color, anchor="w", width=150).pack(side='left', padx=10, pady=6)

    # Placeholder methods for functionality (implement as needed)
    def update_status(self, text, color="#8b949e"):
        self.status_text.set(text)
        self.status_label.configure(text_color=color)

    def refresh_available_pairs(self):
        # Implement pair detection logic here
        pairs = ["EUR/USD", "GBP/USD", "USD/JPY", "AUD/USD", "USD/CHF"]
        if hasattr(self, 'pair_combo'):
            self.pair_combo.configure(values=pairs)

    def update_pair_info(self, *args):
        pair = self.selected_pair.get()
        info_text = f"PAIR: {pair}\n\nData Available: 2014-2025\nTotal Days: 3,650"
        if hasattr(self, 'pair_info_label'):
            self.pair_info_label.configure(text=info_text)

    def select_data_folder(self):
        messagebox.showinfo("Info", "Data folder selection - implement file dialog here")

    def start_backtest_thread(self):
        # Show sample results for demo
        sample_metrics = {
            'Total Return (%)': 15.3,
            'Win Rate (%)': 58.2,
            'Profit Factor': 1.42,
            'Sharpe Ratio': 1.18,
            'Sortino Ratio': 1.45,
            'Max Drawdown (%)': -8.7,
            'Avg Win ($)': 125.50,
            'Avg Loss ($)': 88.30,
            'Largest Win ($)': 450.00,
            'Largest Loss ($)': 210.00,
            'Max Consecutive Wins': 7,
            'Max Consecutive Losses': 4
        }

        # Create sample trades data
        import pandas as pd
        import numpy as np
        from datetime import datetime, timedelta

        dates = pd.date_range(start='2024-01-01', end='2024-12-31', freq='D')[:100]
        np.random.seed(42)
        
        sample_trades = pd.DataFrame({
            'exit_time': dates,
            'monetary_pnl': np.random.normal(10, 50, 100),
            'balance': 10000 + np.cumsum(np.random.normal(10, 50, 100))
        })

        summary = "TRADES: 100\nWIN RATE: 58.2%\nP&L: $1,530.00\nPIPS: 890.5\nRETURNS: +15.30%\nFINAL BALANCE: $11,530.00"

        self.update_results_ui(summary, sample_metrics, sample_trades)

    def generate_report(self):
        messagebox.showinfo("Report", "HTML report generation - implement report logic here")

    def clear_cache(self):
        messagebox.showinfo("Cache", "Cache cleared successfully")

    def export_to_csv(self):
        messagebox.showinfo("Export", "Data exported to CSV successfully")

# Example usage
if __name__ == '__main__':
    app = ctk.CTk()
    backtester = BacktesterUI(app)
    app.mainloop()

C:\Users\wolfr\AppData\Local\Temp\ipykernel_10796\3728503821.py:354: DeprecationWarning: trace_variable() is deprecated and not supported with Tcl 9; use trace_add() instead.
  self.selected_pair.trace('w', self.update_pair_info)
findfont: Font family 'Helvetica' not found.
findfont: Font family 'Helvetica' not found.
findfont: Font family 'Helvetica' not found.
findfont: Font family 'Helvetica' not found.
findfont: Font family 'Helvetica' not found.
findfont: Font family 'Helvetica' not found.
findfont: Font family 'Helvetica' not found.
findfont: Font family 'Helvetica' not found.
findfont: Font family 'Helvetica' not found.
findfont: Font family 'Helvetica' not found.
findfont: Font family 'Helvetica' not found.
findfont: Font family 'Helvetica' not found.
findfont: Font family 'Helvetica' not found.
findfont: Font family 'Helvetica' not found.
findfont: Font family 'Helvetica' not found.
findfont: Font family 'Helvetica' not found.
findfont: Font family 'Helvetica' not found.
findf